- Aim: Combining OCR & LLM search to get information from a scanned pdf.
- This notebook tests AWS Textract on a few images (pdf converted to images)

In [ ]:
import boto3
import io
from PIL import Image, ImageDraw

In [ ]:
# Connect to Amazon Textract to detect text in the document
client = boto3.client("textract", region_name="eu-west-2", aws_access_key_id='', 
aws_secret_access_key='')

In [ ]:
def analyze_file(
        client, feature_types, *, document_file_name=None, document_bytes=None
    ):
        """
        Detects text and additional elements, such as forms or tables, in a local image
        file or from in-memory byte data.
        The image must be in PNG or JPG format.

        :param feature_types: The types of additional document features to detect.
        :param document_file_name: The name of a document image file.
        :param document_bytes: In-memory byte data of a document image.
        :return: The response from Amazon Textract, including a list of blocks
                 that describe elements detected in the image.
        """
        
        with open(document_file_name, "rb") as document_file:
            document_bytes = document_file.read()
        response = client.analyze_document(
            Document={"Bytes": document_bytes}, FeatureTypes=feature_types
        )

                  

In [ ]:
analyze_file(client, document_file_name='test.png', feature_types=['FORMS'])